In [18]:
import sys
import psycopg2
sys.path.append('../')
from helper import db
import pandas as pd
import numpy as np

In [41]:
# Based on https://www.usna.edu/Users/oceano/pguth/md_help/html/approx_equivalents.htm rounding latitude and longitude 
# to .01 degree approximates by 11.1km which is ok at our scale
query = """
SELECT
extract(year from data_date) as year,
depth,
ROUND(latitude) as latitude, 
ROUND(longitude) as longitude,
ROUND(AVG(temperature), 3) as temperature,
ROUND(AVG(salinity), 3) as salinity
FROM OCEAN_DATA
WHERE data_date BETWEEN '2009-01-01' AND '2011-12-31'
AND to_char(data_date,'Mon') in ('Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep')
GROUP BY year, depth, latitude, longitude
ORDER BY year, temperature;
"""
#df = db.run_query(query)

In [42]:
df.head()

,year,depth,latitude,longitude,temperature,salinity
0,2009.0,"(0, 100]",55,-55,-1.609,32.720
1,2009.0,"(0, 100]",55,-55,-1.537,33.389
2,2009.0,"(0, 100]",55,-55,-1.534,33.227
3,2009.0,"(0, 100]",55,-54,-1.487,32.855
4,2009.0,"(0, 100]",55,-54,-1.485,32.839


In [446]:
from helper import db
from pandas.api.types import CategoricalDtype
def make_categorical(df):
    #turn string back into categories... yes you have to do it manually apprarently
    intervals = ['(0, 100]', '(100, 200]', '(200, 300]', '(300, 400]', '(400, 500]', '(500, 600]', '(600, 700]', '(700, 800]', '(800, 900]', '(900, 1000]',
            '(1000, 1100]', '(1100, 1200]', '(1200, 1300]', '(1300, 1400]', '(1400, 1500]', '(1500, 1600]', '(1600, 1700]', '(1700, 1800]', '(1800, 1900]', '(1900, 2000]', 
            '(2000, 2100]', '(2100, 2200]', '(2200, 2300]', '(2300, 2400]', '(2400, 2500]', '(2500, 2600]', '(2600, 2700]', '(2700, 2800]', '(2800, 2900]', '(2900, 3000]',
            '(3000, 3100]', '(3100, 3200]', '(3200, 3300]', '(3300, 3400]', '(3400, 3500]', '(3500, 3600]', '(3600, 3700]', '(3700, 3800]', '(3800, 3900]', '(3900, 4000]', 
            '(4000, 4100]', '(4100, 4200]', '(4200, 4300]', '(4300, 4400]', '(4400, 4500]', '(4500, 4600]', '(4600, 4700]', '(4700, 4800]', '(4800, 4900]', '(4900, 5000]',
            '(5000, 5100]', '(5100, 5200]', '(5200, 5300]', '(5300, 5400]', '(5400, 5500]', '(5500, 5600]', '(5600, 5700]', '(5700, 5800]', '(5800, 5900]', '(5900, 6000]',
           ]

    cat_type = CategoricalDtype(categories=intervals, ordered=True)
    df["depth_range"] = df["depth_range"].astype(cat_type)
    return df


def variation(df):
    # Be sure to only input df[df.depth < "(2100, 2200]"]
    for interval in df["depth_range"]:
        # Get the baseline value of the parameter in 2009
        depth = (df.depth_range == interval)
        y2009 = (df.year == 2009)
        baseline_temp = df.loc[depth & y2009]["temperature"].item()
        baseline_sal = df.loc[depth & y2009]["salinity"].item()
        # Use subset to update values
        df.loc[depth, "temp_variation"] = df.loc[depth, "temperature"]- baseline_temp
        df.loc[depth, "sal_variation"] = df.loc[depth, "salinity"]- baseline_sal
    return df


def param_data():
    '''Return a dataframe with average temperature and salinity by depth and year, 
    temperature and salinity evolution compared to 2009 and year over year temperature and salinity evolution.
    The SQL query filters outlier data (e.g. temperature below -2.5 or above 40 degrees celsius) and only takes values
    for the months of April to September (2009-2018) since these are the months the fish live in the Estuary.'''
    
    # Very important to group by the renamed depth column (not 'depth') otherwise we'll have duplicates. 
    # Include 'AND in_gulf = 1' to the filter when the gstpp data is all added.
    query = """
    SELECT
    extract(year from data_date) as year,
    REPLACE(depth, '.0', '') as depth_range,
    ROUND(AVG(temperature), 3) as temperature,
    ROUND(AVG(salinity), 3) as salinity
    FROM OCEAN_DATA
    WHERE data_date BETWEEN '2009-01-01' AND '2018-12-31'
    AND salinity BETWEEN 30 and 41
    AND temperature BETWEEN -2.5 and 40
    AND to_char(data_date,'Mon') in ('Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep')
    AND depth <> 'nan'
    GROUP BY year, depth_range
    ORDER BY year, temperature;
    """
    param_df = db.run_query(query)
    
    # Turn the depth_range column values back into categories
    param_df = make_categorical(param_df)
    
    # Change temp and salinity to floats
    param_df["temperature"] = param_df["temperature"].astype(float)
    param_df["salinity"] = param_df["salinity"].astype(float)
    
    # Calculate the temperature and salinity variation from 2009
    param_df = variation(param_df[param_df.depth_range < "(2100, 2200]"])
    
    # Calculate year-to-year temperature and salinity variation: use diff on the sorted df to calculate difference,
    # then set the year 2009 as 0 since this is our first year.
    param_df.sort_values(by=['depth_range', 'year'], inplace=True)
    param_df['temp_var_ytoy'] = param_df['temperature'].diff()
    param_df['sal_var_ytoy'] = param_df['salinity'].diff()
    param_df.loc[(param_df['year'] == 2009), "temp_var_ytoy"] = 0
    param_df.loc[(param_df['year'] == 2009), "sal_var_ytoy"] = 0
    
    # Make the year column a string
    param_df['year'] = param_df['year'].astype(int).astype(str)

    return param_df.reset_index(drop=True)

In [447]:
# %%time
df = param_data()
df

C:\Users\Kik\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kik\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,year,depth_range,temperature,salinity,temp_variation,sal_variation,temp_var_ytoy,sal_var_ytoy
0,2009,"(0, 100]",7.594,33.282,0.000,0.000,0.000,0.000
1,2010,"(0, 100]",8.408,33.890,0.814,0.608,0.814,0.608
2,2011,"(0, 100]",7.222,34.026,-0.372,0.744,-1.186,0.136
3,2012,"(0, 100]",8.786,33.940,1.192,0.658,1.564,-0.086
4,2013,"(0, 100]",7.947,33.965,0.353,0.683,-0.839,0.025
...,...,...,...,...,...,...,...,...
205,2014,"(2000, 2100]",3.444,34.919,-0.023,-0.007,0.024,-0.007
206,2015,"(2000, 2100]",3.465,34.938,-0.002,0.012,0.021,0.019
207,2016,"(2000, 2100]",3.499,35.005,0.032,0.079,0.034,0.067
208,2017,"(2000, 2100]",3.531,34.912,0.064,-0.014,0.032,-0.093


In [448]:
df.dtypes

year                object
depth_range       category
temperature        float64
salinity           float64
temp_variation     float64
sal_variation      float64
temp_var_ytoy      float64
sal_var_ytoy       float64
dtype: object

In [449]:
df.to_csv('sample_param_data.csv')

In [452]:
df.describe()

,temperature,salinity,temp_variation,sal_variation,temp_var_ytoy,sal_var_ytoy
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,4.912705,34.890843,-0.145771,0.013986,-0.008595,-0.009933
std,1.548156,0.315779,0.482467,0.184837,0.461124,0.192698
min,3.369000,31.783000,-2.217000,-1.499000,-4.760000,-2.609000
25%,3.734250,34.919000,-0.292500,-0.023000,-0.054750,-0.016750
50%,4.191500,34.926500,-0.078500,-0.007000,0.000000,0.000000
75%,5.922250,34.976750,0.000000,0.000750,0.049750,0.008000
max,10.137000,35.188000,2.543000,1.110000,1.564000,0.608000
